Visualize the initial graph

Load the RDF data from the ttl file and define namespaces.

In [2]:
from rdflib import Graph, Namespace, Literal, BNode
from pyvis.network import Network

g = Graph()
g.parse("family_rdf.ttl", format="ttl")
g.serialize(destination="rdfdata.ttl")

prefixes = {
    "foaf": "http://xmlns.com/foaf/0.1/",
    "fam": "http://example.org/family#",
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "ex": "http://example.org/"
}

FOAF = Namespace("http://xmlns.com/foaf/0.1/")
FAM = Namespace("http://example.org/family#")
RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")



Iterate over the triples in the RDF graph, add the given relationships to the graph, visulize the initial graph

In [3]:
def get_prefixed_uri(uri):
    for prefix, base_uri in prefixes.items():
        if uri.startswith(base_uri):
            return uri.replace(base_uri, f"{prefix}:")
    return uri 

def get_label(node):
    if isinstance(node, BNode):
        name = g.value(node, FOAF.name)
        if isinstance(name, Literal):  
            return str(name)
    return get_prefixed_uri(str(node))

initial_graph = Network(notebook=True, directed=True)

for subj, pred, obj in g:
    subj_label = get_label(subj)
    obj_label = get_label(obj)
    pred_label = get_prefixed_uri(str(pred))
    
    initial_graph.add_node(subj_label, label=subj_label)
    initial_graph.add_node(obj_label, label=obj_label)
    initial_graph.add_edge(subj_label, obj_label, label=pred_label, color="black")

initial_graph.show("initial_graph.html")


initial_graph.html


Visulize the entailment graph

In [4]:
subproperty_map = {}
for subprop, _, superprop in g.triples((None, RDFS.subPropertyOf, None)):
    subproperty_map[subprop] = superprop

new_triples = set()
for subj, pred, obj in g:
    if pred in subproperty_map:
        superpred = subproperty_map[pred]
        new_triples.add((subj, superpred, obj))

for subj, superpred, obj in new_triples:
    g.add((subj, superpred, obj))

entailment_graph = Network(notebook=True, directed=True)

for subj, pred, obj in g:
    subj_label = get_label(subj)
    obj_label = get_label(obj)
    pred_label = get_prefixed_uri(str(pred))
    
    color = "blue" if (subj, pred, obj) in new_triples else "black"
    
    entailment_graph.add_node(subj_label, label=subj_label)
    entailment_graph.add_node(obj_label, label=obj_label)
    entailment_graph.add_edge(subj_label, obj_label, label=pred_label, color=color)

entailment_graph.show("entailment_graph.html")

entailment_graph.html
